In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##This cell calculates most of the color features, including number of hues(hue_counts), average hue(mean_hue), major hue(max_hue), average saturation(mean_sat), hue contrast(hues_dis), hue histogram(hues_his), major hue ratio(max_hue_pix_ratio)
##Refer to formula (7)-(10) in my paper or Page 4 in the reference paper
import cv2
import numpy as np
path1=""
hmax=0
hues=[]
hue_counts=[]
max_hue=[]
hues_list=[]
hues_his=[]
hues_dis=[]
max_hue_pix_ratio=[]
mean_hue=[]
mean_sat=[]
def hue_distance(h1,h2):
    return min(abs(h1-h2),abs(180-h1+h2),abs(180-h2+h1))
# files=os.listdir(path)
lp=list1
for i in range(len(lp)):
    # im_path=path+str(i)+".png"
    im_path=path1+list1[i]
    # im_path=lp[i]
    img = cv2.imread(im_path)
    while img.shape[0]>500:
        img=cv2.pyrDown(img)# downsampling, because the color template fitting is too slow on large image
    img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
#     imgs_hls.append(img_hls)
#     if i%30==0:
#         print(i)
    im=img_hls[:,:,0]
    mean_hue.append(np.mean(im))
    mean_sat.append(np.mean(img_hls[:,:,2]))
    hue_h=np.zeros((60,))
    hue_l=[]
    ims=np.int32((img_hls[:,:,2]/255.0)>0.001)
    iml=np.int32((img_hls[:,:,1]/255.0)>0.001)
    h_bins=np.floor(im/3)*3
#     hues.append(h_bins)
    hue_c=0
    max_pix=0
    max_id=-1
    for j in range(60):
        sign=np.multiply(np.int32(h_bins>=j*3), np.int32(h_bins<(j+1)*3))
        pixel_counts=np.sum(np.multiply(np.multiply(ims,sign),iml))
        if pixel_counts>50:
            hue_c=hue_c+1
            hue_h[j]=pixel_counts
            hue_l.append(3*j)
            if pixel_counts>max_pix:
                max_pix=pixel_counts
                max_id=j
    hue_counts.append(hue_c)
    max_hue_pix_ratio.append(max_pix/(im.shape[0]*im.shape[1]))
    max_hue.append(3*max_id)
    hues_his.append(hue_h)
    max_huedis=0
    for k in range(len(hue_l)):
        for l in range(k):
            huedis=hue_distance(hue_l[k],hue_l[l])
            if huedis>max_huedis:
                max_huedis=huedis
    hues_dis.append(max_huedis)
#     for w in range(im.shape[0]):
#         for h in range(im.shape[1]):
#             if im[w][h]>hmax:
#                 hmax=im[w][h]
    if i%30==0:
        print(i)
        print(max_pix/(im.shape[0]*im.shape[1]))

In [ ]:
##This cell calculates the sharpness feature
##Refer to formula (12)-(15) in my paper or Page 6 in the reference paper
blurriness=[]
for i in range(len(list1)):
    # im_path=path+str(i)+".png"
    im_path=path1+list1[i]
    img = cv2.imread(im_path)
    # while img.shape[0]>500:
    #     img=cv2.pyrDown(img)
    grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    FFT=np.absolute(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(grayscale))))
    FFT1=FFT/np.sqrt(FFT.size)
    score_matrix=np.int32(FFT1>5)
    blurriness.append(np.sum(score_matrix)/FFT1.size)
    if i%30==0:
        print(i)

In [ ]:
##This cell defines two helper functions for next cell which calculates the edge features
def compute_thres(im):
  thres=np.mean(im)
  left=0
  right=0
  thres_r=np.max(im)
  thres_l=np.min(im)
  total=np.sum(im)
  rat=np.sum(np.multiply(np.int32(im>thres),im))/total
  pre_loss=1
  bar=0.81
  loss=np.abs(rat-bar)
  sign=np.int32(im>thres)
  while np.abs(rat-bar)>0.00001 and np.abs(pre_loss-loss)>0.0000001:
    if rat<bar:
      thres_r=thres
    elif rat>bar:
      thres_l=thres
    else:
      break
    thres=thres_l+0.5*(thres_r-thres_l)
    # print(thres_r)
    # print(thres)
    # print(thres_l)
    # print("\n")
    pre_loss=loss
    rat=np.sum(np.multiply(np.int32(im>thres),im))/total
    loss=np.abs(rat-bar)
    # print(loss)
    sign=np.int32(im>thres)
  return np.sum(sign)/(im.shape[0]*im.shape[1]),sign
def get_bbx1(L_abs):
  startX=0
  startY=0
  endX=sign.shape[0]-1
  endY=sign.shape[1]-1
  start_point=0
  end_point=0
  bar=0.81
  total=bar*np.sum(L_abs)
  # Y=sign.shape[0]-1
  # X=sign.shape[1]-1
  step=(int)(min(L_abs.shape[0],L_abs.shape[1])/100.0)+1
  while np.sum(L_abs[startX:endX,startY:endY])>bar*total:
    current=np.sum(L_abs[startX:endX,startY:endY])
    x1=(endY-startY)/(current-np.sum(L_abs[startX+step:endX,startY:endY])+0.001)
    x2=(endY-startY)/(current-np.sum(L_abs[startX:endX-step,startY:endY])+0.001)
    y1=(endX-startX)/(current-np.sum(L_abs[startX:endX,startY+step:endY])+0.001)
    y2=(endX-startX)/(current-np.sum(L_abs[startX:endX,startY:endY-step])+0.001)
    Lxy=[x1,x2,y1,y2]
    max_id=0
    max_v=x1
    for k in range(4):
      if Lxy[k]>max_v:
        max_id=k
        max_v=Lxy[k]
    if max_id==0:
      startX=startX+step
    elif max_id==1:
      endX=endX-step
    elif max_id==2:
      startY=startY+step
    else:
      endY=endY-step

  return startX, endX, startY, endY

In [ ]:
##This cell calculates the edge features, including bound box area of 81% edge energy(bbx_area), exact area of the region with 81% edge energy(edge_area), the ratio of pixel with above average edge energy(mean_ratios)
##Refer to formula (17)-(19) in my paper or Page 6-7 in the reference paper
Laps=[]
mean_ratios=[]
edge_area=[]
bbx_area=[]
edge_disX=[]
edge_disY=[]
ddepth = cv2.CV_64F
# ddepth = cv2.CV_16S
kernel_size = 3
for i in range(len(list1)):
    # im_path=path+str(i)+".png"
    im_path=path1+list1[i]
    img = cv2.imread(im_path)
#     while img.shape[0]>500:
#         img=cv2.pyrDown(img)# downsampling, because the color template fitting is too slow on large image
    # grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # imgs2.append(img)
    im_B=img[:,:,0]
    im_G=img[:,:,1]
    im_R=img[:,:,2]
    blur_G = cv2.GaussianBlur(im_G,(5,5),0)
    blur_B = cv2.GaussianBlur(im_B,(5,5),0)
    blur_R = cv2.GaussianBlur(im_R,(5,5),0)
    LG = cv2.Laplacian(blur_G, ddepth, ksize=kernel_size)
    LB = cv2.Laplacian(blur_B, ddepth, ksize=kernel_size)
    LR = cv2.Laplacian(blur_R, ddepth, ksize=kernel_size)
    L=(LG+LB+LR)/3.0
    L_abs=cv2.convertScaleAbs(L)
    # Laps.append(L_abs)
    mean_ratio=np.mean(np.int32(L_abs>np.mean(L_abs)))
    mean_ratios.append(mean_ratio)
    area,sign=compute_thres(L_abs)
    edge_area.append(area)
    # startX, endX, startY, endY=get_bbx(sign)
    startX, endX, startY, endY=get_bbx1(L_abs)
    edge_disX.append((endX-startX)/sign.shape[0])
    edge_disY.append((endY-startY)/sign.shape[1])
    bbx_area.append((endX-startX)*(endY-startY)/(sign.shape[0]*sign.shape[1]))
    # print(i)
    if i%30==0:
        print(i) 

In [ ]:
##This cell calculates the composition features, including average x positions(Xs), average y positions(Ys), spatial variance(sqs) and skewness(sks) of three largest regions, 
##Each element in Xs contains three numbers corresponding to three regions
##Refer to formula (24)-(27) in my paper or Page 7 in the reference paper
Xs=[]
Ys=[]
X_sqs=[]
Y_sqs=[]
sqs=[]
sks=[]
for i in range(len(list1)):
    # im_path=path+str(i)+".png"
    im_path=path1+list1[i]
    img = cv2.imread(im_path)
    while img.shape[0]>256:
        img=cv2.pyrDown(img)# downsampling, because the color template fitting is too slow on large image
    # imgs1.append(img)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    twoDimage = img.reshape((-1,3))
    twoDimage = np.float32(twoDimage)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 6
    attempts=10
    ret,label,center=cv2.kmeans(twoDimage,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((img.shape))
    sign=np.int32(label.reshape((img.shape[0],img.shape[1]))==0)
    weight=np.sum(sign)
    weights=[weight]
    w_ids=[0]
    signs=[np.int32(label.reshape((img.shape[0],img.shape[1]))==0)]
    for k in range(1,center.shape[0]):
      sign=np.int32(label.reshape((img.shape[0],img.shape[1]))==k)
      weight=np.sum(sign)
      insert_id=len(weights)
      for u in range(len(weights)):
        if weights[len(weights)-u-1]>=weight:
          insert_id=len(weights)-u
          break
        else:
          insert_id=len(weights)-u-1
      weights.insert(insert_id, weight)
      w_ids.insert(insert_id, k)
      signs.insert(insert_id, sign)
    # WS.append(weights)
    meanX=[]
    meanY=[]
    varX=[]
    varY=[]
    var=[]
    skew=[]
    for j in range(3):
      wid=w_ids[j]
      sign=signs[j]#np.int32(label.reshape((img.shape[0],img.shape[1]))==wid)
      x_ids, y_ids=np.nonzero(sign)
      x_ids=np.array(x_ids)/(img.shape[0])
      y_ids=np.array(y_ids)/(img.shape[1])
      x_c=np.sum(x_ids)/(weights[j])
      y_c=np.sum(y_ids)/(weights[j])
      x_sq=np.sum(np.square(x_ids-x_c))/(weights[j])
      y_sq=np.sum(np.square(y_ids-y_c))/(weights[j])
      sq=np.sum(np.square(x_ids-x_c)+np.square(y_ids-y_c))/(weights[j])
      sk=np.sum(np.power(y_ids-y_c,3)+np.power(x_ids-x_c,3))/(weights[j])
      var.append(sq)
      skew.append(sk)
      meanX.append(x_c)
      meanY.append(y_c)
      varX.append(x_sq)
      varY.append(y_sq)

    Xs.append(meanX)
    Ys.append(meanY)
    X_sqs.append(varX)
    Y_sqs.append(varY) 
    sqs.append(var) 
    sks.append(skew)    
    if i%30==0:
      print(i)

In [ ]:
##This cell calculates the color and brightness features for the focus region
##Refer to the last paragraph of color features in my paper or formula (23)-(25) in Page 8 in the reference paper
focus_h=[]
focus_l=[]
focus_s=[]


for i in range(len(list1)):
    # im_path=path+str(i)+".png"
    im_path=path1+list1[i]
    img = cv2.imread(im_path)
    while img.shape[0]>500:
        img=cv2.pyrDown(img)# downsampling, because the color template fitting is too slow on large image
    img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    fr_x0=(int)((1/3.0)*img_hls.shape[0]-(1/20.0)*img_hls.shape[0])
    fr_x1=(int)((2/3.0)*img_hls.shape[0]+(1/20.0)*img_hls.shape[0])
    fr_y0=(int)((1/3.0)*img_hls.shape[1]-(1/20.0)*img_hls.shape[1])
    fr_y1=(int)((2/3.0)*img_hls.shape[1]+(1/20.0)*img_hls.shape[1])
    fr=img_hls[fr_x0:fr_x1,fr_y0:fr_y1,:]
    fr_h=np.mean(fr[:,:,0])
    fr_l=np.mean(fr[:,:,1])
    fr_s=np.mean(fr[:,:,2])
    focus_h.append(fr_h)
    focus_l.append(fr_l)
    focus_s.append(fr_s)
#     imgs_hls1.append(img_hls)
    if i%30==0:
        print(i)

In [ ]:
##This cell calculates the entropy features
##Refer to formula (16) in my paper
import scipy
import scipy.special
mean_entr=[]

for i in range(len(list1)):
    # im_path=path+str(i)+".png"
    im_path=path1+list1[i]
    im = cv2.imread(im_path)
    while im.shape[0]>500:
        im=cv2.pyrDown(im)# downsampling, because the color template fitting is too slow on large image
    img=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    win_size=10
    entrs=[]
    for w in range((int)((img.shape[0])/win_size)):
        for h in range((int)((img.shape[1])/win_size)):
            x0=w*win_size
            x1=(w+1)*win_size
            y0=h*win_size
            y1=(h+1)*win_size
            patch=img[x0:x1,y0:y1]
            vals=np.unique(patch)
            fr=np.zeros((vals.shape[0],))
            for j in range(vals.shape[0]):
                fr[j]=np.sum(np.int32(patch==vals[j]))/100
            entr=scipy.special.entr(fr)
            entrs.append(np.sum(entr))
    entrs=np.sort(np.array(entrs))
    entrs=entrs[(int)(entrs.shape[0]*0.2):(int)(entrs.shape[0]*0.8)]
    mean_entr.append(np.mean(entrs))
    
    if i%30==0:           
        print(i)

In [ ]:
##This cell calculates the brightness features, including average brightness(aver_brightness) and brightness contrast(bright_contrasts)
##Refer to formula (5)-(6) in my paper or Page 6 in the reference paper
aver_brightness=[]
bright_contrasts=[]
for i in range(len(list1)):
    # im_path=path+str(i)+".png"
    im_path=path1+list1[i]
    img = cv2.imread(im_path)
    while img.shape[0]>500:
        img=cv2.pyrDown(img)# downsampling, because the color template fitting is too slow on large image
    img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    iml=img_hls[:,:,1]
    aver_brightness.append(np.mean(iml))
    im_his,bin_edges=np.histogram(iml,bins=40,density=False,range=(0,255))
    max_bin=0
    max_count=0
    total_count=np.sum(im_his)
    cumulative=0
    for j in range(im_his.shape[0]):
        if im_his[j]>max_count:
            max_count=im_his[j]
            max_bin=j
    cumulative=max_count
    left_end=max_bin
    right_end=max_bin
    while cumulative<total_count*0.98:
        if left_end-1>=0:
            left_end=left_end-1
            cumulative=cumulative+im_his[left_end]
        if right_end+1<im_his.shape[0] and cumulative<total_count*0.98:
            right_end=right_end+1
            cumulative=cumulative+im_his[right_end]  
    contrast=bin_edges[right_end]-bin_edges[left_end]
    bright_contrasts.append(contrast)
    if i%30==0:
        print(i)